# AdmixtureBayes manual

AdmixtureBayes is a program for estimating admixture graphs to describe the history of a number of populations. Once installed, it can be run through command line using the command `AdmixtureBayes run`. The input file is of the form


In [1]:
f=open('big_allele_counts.txt','r')
print '\n'.join(f.readlines()[:10])
f.close()

s1 s2 s3 s4 out

9,11 13,7 11,9 14,6 14,6

4,16 4,16 0,20 1,19 2,18

0,20 0,20 0,20 1,19 1,19

9,11 7,13 9,11 6,14 6,14

15,5 18,2 20,0 18,2 18,2

1,19 3,17 1,19 0,20 0,20

7,13 13,7 14,6 12,8 17,3

0,20 0,20 0,20 0,20 2,18

0,20 0,20 0,20 0,20 2,18



The line has a column for each population. Every line is a bi-allelic SNP where `x,y` represents the counts of the two genetic types. 

# Basic AdmixtureBayes

AdmixtureBayes and a full AdmixtureBayes post-analysis can be run with the 6 commands

```{bash}
$ AdmixtureBayes run --input_file big_allele_counts.txt --outgroup out
$ AdmixtureBayes posterior --input_file result_mc3.csv --covariance_matrix_file covariance_and_multiplier.txt 
$ AdmixtureBayes plot --plot top_trees --posterior_distribution_file posterior_distributions.csv
$ AdmixtureBayes plot --plot estimates --posterior_distribution_file posterior_distributions.csv
$ AdmixtureBayes plot --plot consensus_trees --posterior_distribution_file posterior_distributions.csv
$ AdmixtureBayes plot --plot top_node_trees --posterior_distribution_file posterior_distributions.csv
```

### AdmixtureBayes run
The command `AdmixtureBayes run` takes as input the above mentioned file `big_alleles.txt` and the name of one of the populations in the dataset that should act as an outgroup. In the graph there will be an uninterrupted line between the outgroup and the root population. This restriction fixes the position of the root in the graph. The command creates a file called `result_mc3.csv` (which could be changed with the flag `--result_file foobar.csv`). It has the following structure

In [2]:
f=open('result_mc3.csv','r')
print '\n'.join(f.readlines()[:2])
f.close()

,add,admixtures,average_branch_length,average_root,descendant_sets,ghost_pops,iteration,likelihood,max_root,mhr,min_root,no_admixes,posterior,prior,proposal_type,rescale_adap_param,rescale_add_adap_param,rescale_admixtures_adap_param,rescale_constrained_adap_param,sliding_regraft_adap_param,total_branch_length,tree,origin,layer

0,0.0,;,0.03359418017669876,0.06718836035339751,s1-s1.s2.s3-s1.s2.s3.s4-s2-s2.s3-s3-s4,0,0,-1389.256827308971,0.06718836035339751,1.0,0.06718836035339751,0,-1394.908024213796,-5.651196904824884,rescale_constrained,,,,0.1,,0.20156508106019255,w.c.1.w-c.0.w-c.0;0.022396120-0.022396120-0.044792240-0.022396120-0.022396120-0.067188360;,0,0



each line in the file represents the MCMC at that certain iteration. There routines `AdmixtureBayes posterior` and `AdmixtureBayes plot` translates this file into more interpretable information. 

### Convergence of AdmixtureBayes run
The default settings terminate the algorithm quickly, so only small uncomplicated datasets will converge. The computation time needed to converge depends mainly on the size of posterior graph space which is a determined by the number of populations and the number of admixture events. The number of SNPs does increase the peakiness of the posterior distribution which increases computation time to a certain extent. 

All AdmixtureBayes runs would eventually converge if given enough time. Therefore, it is important to control the length of the algorithm. The length is set with the parameter `--n` which is $200$ by default. With aroung 10 populations a better value is $20000$. One can also set a stopping criteria, that stops the chain when a convergence condition has been fulfilled. It is turned on by declaring the flag `--stop_criteria`. One still has to set a maximum number of iterations using `--n`. Running AdmixtureBayes with this option could look like this

```
$ AdmixtureBayes run --input_file big_allele_counts.txt --outgroup out --stop_criteria --n 1000000
```
This requires that `R` is installed with the packages `coda` and `rwty`. (AdmixtureBayes will call the stop_criteria check with the command line command `Rscript ESS.R [...]`. On some systems R may not be called using `Rscript`. This can be changed by setting `--Rscript_command R`)

### AdmixtureBayes posterior
The routine `AdmixtureBayes posterior` discards burn-in of the MCMC, thins the chain, and transforms the sampled admixture graphs into a unique format called `string_tree`. To do this it needs to know the populations of the dataset - these are available in the file `covariance_and_multiplier.txt` generated by `AdmixtureBayes run`. The output of `AdmixtureBayes posterior` looks like this

In [3]:
f=open('posterior_distributions.csv','r')
print '\n'.join(f.readlines()[:3])
f.close()

pops,string_tree,no_admixes,topology

s1-s1.s2-s1.s2.s3-s2-s2.s3-s3,s1=s2=s3=w.a.w-c.0.c.2-c.0;0.002586817-0.077780913-0.001668333-0.050026384-0.031787628-0.017594094-0.056015726;0.459,1,w.a.w-c.0.c.2-c.0

s1-s1.s2-s1.s2.s3-s2-s2.s3-s3,s1=s2=s3=w.a.w-c.0.c.2-c.0;0.004636805-0.078315185-0.010039090-0.050147478-0.015929088-0.018018536-0.056317524;0.47,1,w.a.w-c.0.c.2-c.0



The file has 4 columns (by default). Each line represents a single particle visited by the MCMC. The first column (`pops`) is a string describing all nodes in the graph. Each node is annotated by the set of populations having the node as ancestor. The string `pops` string contains all unique annotations found in the graph. The `string_tree` represents the full admixture graph. It has the format *population1=population2=...=populationN,topology;branch1-branch2-...-branchB;admixture_proportion1-...-admixture_proportionK*. The `no_admixes` column is the number of admixtures in the graph and `topology` is the string representation of the shape of the admixture graph representation. The topologies can be visualized by `AdmixtureBayes plot` which can do it in 4 different ways.

### AdmixtureBayes plot --plot top_trees
This will plot the 3 most probable topologies of the topology column in the file `posterior_distributions.csv`. The number three can be set by the flag `--top_trees_to_plot X` where X is the desired number of top topologies to plot. Depending on the seed the most probable topology of the dataset in `big_allele_counts.txt` may look like this:

![title](topology_1.png)

It is also possible to get the probability of each topology using the flag `--write_rankings_to_file trankings.txt` (you don't have to name the file `trankings.txt`). It looks like this:

In [4]:
f=open('trankings.txt','r')
print '\n'.join(f.readlines())
f.close()

w.a.w.w-c.0.c.w.2-c.0.w-c.0,0.677165354331

w.a.w.w-c.0.c.w.2-w.c.1-c.0,0.283464566929

w.a.w.w-c.0.c.w.2-c.w.0-c.0,0.0393700787402



Hence, the probability of the true graph having the topology plotted above is 67.7%.

### AdmixtureBayes plot --plot estimates
This option calculates the 95% posterior intervals of the branch lengths and admixture proportions for each of the 3 most probable topologies. Those posterior intervals are saved in files named `topology_estimates_branches_1.txt`, `topology_estimates_admixtures_1.txt`, ..., `top_estimates_admixtures_3.txt`. The number three can be changed to X by the flag `--estimate_topologies X`. Each branch and admixture node is labeled - the labeling can be seen in the plots created (`topology_labels_1.png`,...,`topology_labels_3.png`). The plots look like this:

![title](topology_labels_1.png)

and the files look like this:

In [5]:
f=open('topology_estimates_branches_1.txt','r')
print '\n'.join(f.readlines())
f.close()

branch label,lower 95%,mean,upper 95%

c1,0.000138610125,0.00502373887209,0.01272741575

c2,0.077774852725,0.0840798772674,0.089194435

c3,0.00050399075,0.0145108733547,0.037891239

c4,0.015400208,0.0313112102733,0.05011417715

c5,0.00054089755,0.0256591066105,0.095231356275

c6,0.0079560652,0.012850887093,0.018187926

c7,0.054619993625,0.0741647628605,0.091567301775

c8,0.000219348475,0.00249501558721,0.00488839365

c9,0.068773468925,0.071678049093,0.075136887025



### AdmixtureBayes plot --plot top_node_trees
A *node tree* is a summary of the admixture graph topology that contains the information from the `pops` column of the file `posterior_distributions.csv`. It is the set of nodes, where a node is encoded as its set of population descendants. A tree with admixture events will often have several nodes with identical descendants and the existence of all of these is lost by the node tree summary. It is, however, possible to reconstruct a minimal topology that contains the node sets of the node tree and only the node sets of the node tree. Two illustratinos (unrelated to the dataset `big_allele_counts.txt`) are shown here:

![title](topological_set_distance2-1.png)

The three most probable node trees are plotted as such a minimal topology by the `--plot top_node_trees` option. Again, the number of three can be changed to any number X; this time by the command `--top_node_trees_to_plot X`. The most probable node tree may look like this;

![title](minimal_topology_1.png)

As an extra feature, each node in the minimal topology is annotated with the probability of that particular node across all node trees (the feature can be turned off with the flag `--dont_annotate_node_posterior`). That means that the posterior probability that a the true tree has a node which is ancestor to s1 and s2 is 100%. A list of the most probable node trees can be constructed with the command `--write_rankings_to_file nrankings.txt` and looks like this

In [6]:
f=open('nrankings.txt','r')
print '\n'.join(f.readlines())
f.close()

s1-s1.s2-s1.s2.s3-s1.s2.s3.s4-s2-s2.s3-s3-s4,0.677165354331

s1-s1.s2-s1.s2.s3.s4-s2-s2.s3-s2.s3.s4-s3-s4,0.283464566929

s1-s1.s2-s1.s2.s3.s4-s1.s2.s4-s2-s2.s3-s3-s4,0.0393700787402



### --plot consensus_trees
The consensus tree is our attempt at summarizing the full sample of admixture graphs into a single graph. It consists of all nodes (annotated by their descendants) which has a posterior probability higher than some threshold. Such a set can be plotted as minimal topology. The option plots the consensus tree for threshold supplied by the flag `--consensus_thresholds X Y Z .. `. The default thresholds are `[0.25, 0.5, 0.75, 0.9, 0.95, 0.99]`.  For this dataset, the consensus tree for threshold 0.25 is
![title](consensus_25.png)
and threshold 90
![title](consensus_90.png)

A list of all nodes and their posterior probabiltity can be constructed by setting `--write_rankings_to_file rankings.txt` which has the content:


In [7]:
f=open('rankings.txt','r')
print '\n'.join(f.readlines())
f.close()

s1.s2,1.0

s3,1.0

s2,1.0

s1,1.0

s2.s3,1.0

s4,1.0

s1.s2.s3.s4,1.0

s1.s2.s3,0.677165354331

s2.s3.s4,0.283464566929

s1.s2.s4,0.0393700787402



# Advanced AdmixtureBayes
There are a lot of options of AdmixtureBayes that are set to reasonable default values. Here I will describe many of those. When running bigger datasets with more than 4-5 populations one should almost always run the program for longer than the default time and increase the number of parallel chains. In those cases, it is important to check for convergence and analyze any problems, so I will also explain how one can do that.

## Covariance estimation
Before the MCMC starts, AdmixtureBayes will estimate an empirical covariance matrix, $\text{cov}_{\text{emp}}$, the degrees of freedom, $\text{df}$ and a bias correction $V$.The model is
$$
\text{cov}_{\text{emp}}\cdot s(\text{cov}_{\text{emp}})\sim W\Bigl( \bigl(\text{cov}(\text{admixture graph})+V\bigr)\cdot s(\text{cov}_{\text{emp}}), \text{df}\Bigr)
$$
where $W$ is the Wishart distribution and $s(\text{cov}_{\text{emp}})$ is a simple normalizing constant that depends on the estimated covariance matrix. The degrees of freedom is estimated by resampling chunks of the input data set (such as `big_allele_counts.txt`). The size of the chunks can be set by the flag `--bootstrap_blocksize`. Its default value is 1000 SNPs and it should reflect the amount of linkage disequilibrium. The number of bootstrap samples is 100 by default and can be set by `--no_bootstrap_samples` (which should be read as Number Of Bootstrap Sample). 

### User defined covariance matrices

It is also possible to use a user defined covariance matrix. To do that, make a plain text file encoded like the AdmixtureBayes-generated file `covariance_without_reduce_name.txt`

In [8]:
f=open('covariance_without_reduce_name.txt', 'r')
print f.read()
f.close()

s1 s2 s3 s4
s1 0.21927528508929103 0.08631942143331606 0.0770408729942629 0.07342254933662819
s2 0.08631942143331606 0.16089727042636967 0.1095959148440056 0.07883753574224335
s3 0.0770408729942629 0.1095959148440056 0.22956850250089256 0.07724944739961169
s4 0.07342254933662819 0.07883753574224335 0.07724944739961169 0.19651926622421673



The $i,j$'th entry in the covariance matrix should estimate $E[(X_i-X_0)(X_j-X_0)/\sqrt{X_R(1-X_R)}]$ where $X_i, X_j, X_0$ and $X_r$ are the SNP frequency in population $i$, population $j$, the outgroup population, and the root population, respectively. This is different from Treemix which uses the formula
$$
E[(X_i-\bar X)(X_j-\bar X)]
$$
where $\bar X$ is the average allele frequency. Notice that in contrast to AdmixtureBayes, Treemix ignores the normalizing factor $\sqrt{X_R(1-X_R)}$. As a consequence, an estimated branch lengths from Treemix is not the exact amount of drift, though it is proportional to the amount of drift. It is also possible to supply a covariance matrix to AdmixtureBayes that ignores the normalizing $\sqrt{X_R(1-X_R)}$, but the branch lengths estimated would likewise not be the exact drift. 

The matrix in the file above has to be invertible and without the outgroup. When using a custom covariance matrix, there is no SNP dataset that AdmixtureBayes can use to compute $\text{df}$ and $\text{V}$. These can be supplied manually with the commands `--wishart_df X`, where $X$ is the desired number of degrees of freedom, and `--variance_correction_input_file foobar.txt` where `foobar.txt` is a plain text file of the format (scientific notation of numbers optional)

In [9]:
f=open('variance_correction.txt', 'r')
print f.read()
f.close()

9.729722412909951046e-02 5.106279687729089328e-02 5.106279687729089328e-02 5.106279687729089328e-02
5.106279687729089328e-02 1.003249366984472618e-01 5.106279687729089328e-02 5.106279687729089328e-02
5.106279687729089328e-02 5.106279687729089328e-02 9.678481241835584781e-02 5.106279687729089328e-02
5.106279687729089328e-02 5.106279687729089328e-02 5.106279687729089328e-02 9.856973513899161543e-02



The variance correction input file does not include the column names and row names, so it should have the same order as the matrix in `covariance_without_reduce_name.txt`. The variance correction can also be set to 0 easily using `--variance_correction None`. 

When applying a user defined covariance matrix, it is necessary to tell it what type of covariance matrix it has been given. In the example above that is done by `--covariance_pipeline 8 9`. So a complete call with a user-defined covariance matrix could look like this

```
$ AdmixtureBayes run --input_file user_defined_covariance.txt --outgroup out --wishart_df 1000.0 --variance_correction None --covariance_pipeline 8 9 
```


### Advanced user defined covariance matrix
As indicated in the previous section, the covariance matrix can be supplied in different stages. The stages are 

* Stage 6: the input data set (such as `big_allele_counts.txt`)
* Stage 7: the covariance matrix **with** the outgroup. As long as the $i,j$'th entry estimates $E[(X_i-\tilde X)(X_j-\tilde X)/\sqrt{X_R(1-X_R)}]$, where $\tilde X$ is any (fixed) linear combination of the $X_k$'s, the subsequent calculations will be the equivalent. This means that the Treemix covariance matrix will fit into stage 7 (but without the normalizing constant for $\sqrt{X_R(1-X_R)}$. 
* Stage 8: the covariance matrix without the outgroup as presented earlier.
* Stage 9: the covariance matrix from stage 8 multiplied by $s(\text{cov}_{\text{emp}})$. 

If someone has a treemix covariance file saved in the plain-text file `outstem.cov` one can run AdmixtureBayes (without the $X_R$ normalizing constant) using the command

```{bash}
$ AdmixtureBayes run --input_file outstem.cov --covariance_pipeline 7 8 9 --wishart_df 1000.0 --variance_correction None --outgroup out
```

The options can also be used to restart an AdmixtureBayes run. By default, AdmixtureBayes makes a covariance file `covariance_and_multiplier.txt`, a degrees of freedom file `DF.txt` and a variance correction file `variance_correction.txt`. Therefore AdmixtureBayes can be restarted using the command

```{bash}
AdmixtureBayes run --input_file covariance_and_multiplier.txt --covariance_pipeline 9 --df_file DF.txt --variance_correction_input_file variance_correction.txt --outgroup out
```

There is also an option to estimate the degrees of freedom using bootstrapped user defined covariance matrices. Say, you have estimated bootstrapped covariance matrices and saved them in files `cboot1.txt`,`cboot2.txt`,...,`cboot100.txt` using the same format as the previously mentioned `covariance_without_reduce_name.txt`. Then AdmixtureBayes can estimate the degrees of freedom using the command

```
$ AdmixtureBayes run --input_file user_defined_covariance.txt --outgroup out --wishart_df 1000.0 --variance_correction None --covariance_pipeline 8 9 --load_bootstrapped_covariances cboot1.txt cboot2.txt cboot3.txt cboot4.txt cboot5.txt cboot6.txt cboot7.txt cboot8.txt cboot9.txt cboot10.txt cboot11.txt cboot12.txt cboot13.txt cboot14.txt cboot15.txt cboot16.txt cboot17.txt cboot18.txt cboot19.txt cboot20.txt cboot21.txt cboot22.txt cboot23.txt cboot24.txt cboot25.txt cboot26.txt cboot27.txt cboot28.txt cboot29.txt cboot30.txt cboot31.txt cboot32.txt cboot33.txt cboot34.txt cboot35.txt cboot36.txt cboot37.txt cboot38.txt cboot39.txt cboot40.txt cboot41.txt cboot42.txt cboot43.txt cboot44.txt cboot45.txt cboot46.txt cboot47.txt cboot48.txt cboot49.txt cboot50.txt cboot51.txt cboot52.txt cboot53.txt cboot54.txt cboot55.txt cboot56.txt cboot57.txt cboot58.txt cboot59.txt cboot60.txt cboot61.txt cboot62.txt cboot63.txt cboot64.txt cboot65.txt cboot66.txt cboot67.txt cboot68.txt cboot69.txt cboot70.txt cboot71.txt cboot72.txt cboot73.txt cboot74.txt cboot75.txt cboot76.txt cboot77.txt cboot78.txt cboot79.txt cboot80.txt cboot81.txt cboot82.txt cboot83.txt cboot84.txt cboot85.txt cboot86.txt cboot87.txt cboot88.txt cboot89.txt cboot90.txt cboot91.txt cboot92.txt cboot93.txt cboot94.txt cboot95.txt cboot96.txt cboot97.txt cboot98.txt cboot99.txt cboot100.txt
```

## Advanced Convergence
### Stop criteria
In the basic chapter, I introduced `--stop_criteria` that will automatically halt the execution of the program when a condition is fulfilled. The condition is that the Effective Sample Size (computed by R package `coda`) of certain summaries exceed a certain threshold. Some summaries are classified as continuous

* The number of admixture events in the graph.
* The average branch length.
* The length of the branch from the outgroup to the root.
* The set distance to a randomly elected node tree.

while some are classified as topological calculated by the `rwty` package

* The path distance to a 0-tree of a randomly elected admixture graph.
* The path distance to a random-tree of a randomly elected admixture graph.
* The path distance to a mode-tree of a randomly elected admixture graph.

For how this works exactly, I refer to the files `ESS.R`, `tree_statistics.py`. The convergence criteria is fulfilled when all the 4 continuous ESS's are larger than the threshold 200 and all topologial ESS's are larger than 200. It is possible to increase these thresholds with `--stop_criteria_continuous_ess_threshold` and `--stop_criteria_topological_ess_threshold`. If the threshold is increased, we are more certain that a stopped chain has converged, but it comes at a price of bigger computation time. Generally it takes a longer time for the topological ESS's to reach 200 than for the continuous. My intuition is that the continuous ESS's measure whether the continuous parameters of the topologies actually found are searched thoroughly, whereas the topological ESS's measure if all equivalent representations of the topology are traversed equally much. The latter is not as important for inference, so it may be useful to have a lower topological ESS threshold than continuous threshold in order not to waste computation time.

By default, the stop criteria will be computed every 200,000 step (which corresponds to `--n=4000` for `--m=50`). The longer, the MCMC chain is run, the larger is the chance of a false positive accepted stop criteria. Therefore, it may be a good idea to increase the distance between each check. If one wants 2,000,000 steps between each check one can write `--stop_criteria_frequency 2000000`.

### MCMCMC
#### Theory and notation
AdmixtureBayes implements an MCMCMC algorithm, which means that $k$ MCMC chains are run in parallel. If $L(x)$ and $p(x)$ is the likelihood and prior, respectively, of graph $x$, the target distribution of the $j$'th chain can be written
$$
L(x)^{1/\beta_j}p(x)
$$
where $1=\beta_1<\beta_2<\cdots<\beta_k$ are fixed "temperatures". This means that the $j$, the higher temperature, and thereby the more does the target distribution look like the prior. (In normal MCMCMC algorithms, both the likelihood and the prior are raised to the power of $1/\beta_j$, but that would lead to an improper distribution here). Notice that the target distribution of the coldest chain is our desired target distribution. Inference of an MCMCMC is done by only looking at the admixture graphs of this chain. At certain times the algorithm proposes to exchange the graphs of two different chains. For example the acceptance probability that chain $i$ and chain $j$ exchange state $x_i$ and $x_j$ is

$$
\frac{L(x_i)^{1/\beta_j}p(x_i)L(x_j)^{1/\beta_i}p(x_j)}{L(x_j)^{1/\beta_j}p(x_j)L(x_i)^{1/\beta_i}p(x_i)}
$$

The idea is that chains with higher temperatures is less peaky and can thereby jump between different modes of the likelihood. The effectiveness of this algorithm is determined by how often a proposed exchange is accepted and how good the 'hottest' chain is at jumping between modes. 
#### Arguments in AdmixtureBayes
The number of chains in the MCMCMC can be set by `--MCMC_chains`. It is 8 by default, but it will often need to be increased to 16-32 (assuming 100,000-1,000,000 SNPs).  

